In [2]:
import pandas as pd 
import dask.dataframe as dd
import pyarrow as pa
import numpy as np
import tqdm
from datetime import datetime as dt
from tsfresh.feature_extraction import extract_features
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot
import scipy
import seaborn
import jieba
# jieba.enable_parallel(4)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.naive_bayes import MultinomialNB, ComplementNB
import lightgbm as lgbm

In [3]:
def binaryRating(rating):
    return 1 if rating > 3 else 0

In [227]:
collection = MongoClient('10.160.1.15:27017')['googleplay']['comments']

In [228]:
df = pd.DataFrame(list(collection.find({},{'comments':1,'ratings':1})))

In [10]:
# df.to_csv("google_play.csv")
df = pd.read_csv("google_play.csv")

In [6]:
# df = df[['comments','ratings']]

In [11]:
wordCount = df.apply(lambda x:len(str(x['comments']).strip()),axis=1)

In [13]:
df = df[(wordCount>=4) & (df['ratings']!=3)]

In [14]:
cut = df[['comments']].apply(lambda x: " ".join(jieba.cut(x['comments'].strip() + " $ENDING$",cut_all=False)),axis=1)
# cut = df[['comments']].apply(lambda x: " ".join(x['comments']) + " $ENDING$",axis=1)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.548 seconds.
DEBUG:jieba:Loading model cost 0.548 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [17]:
from gensim.corpora import WikiCorpus

In [18]:
# wiki_corpus = WikiCorpus('zhwiki-20190320-pages-articles-multistream.xml.bz2', dictionary={})

In [19]:
texts_num = 0
with open("wiki_zh_tw.txt",'w',encoding='utf-8') as output:
    for text in df['comments']:
        output.write(text + '\n')
        texts_num += 1
        if texts_num % 1000 == 0:
            print("已處理 %d 篇文章" % texts_num)
    output.close

已處理 1000 篇文章
已處理 2000 篇文章
已處理 3000 篇文章
已處理 4000 篇文章
已處理 5000 篇文章
已處理 6000 篇文章
已處理 7000 篇文章
已處理 8000 篇文章
已處理 9000 篇文章
已處理 10000 篇文章
已處理 11000 篇文章
已處理 12000 篇文章
已處理 13000 篇文章
已處理 14000 篇文章
已處理 15000 篇文章
已處理 16000 篇文章
已處理 17000 篇文章
已處理 18000 篇文章
已處理 19000 篇文章
已處理 20000 篇文章
已處理 21000 篇文章
已處理 22000 篇文章
已處理 23000 篇文章
已處理 24000 篇文章
已處理 25000 篇文章
已處理 26000 篇文章
已處理 27000 篇文章
已處理 28000 篇文章
已處理 29000 篇文章
已處理 30000 篇文章
已處理 31000 篇文章
已處理 32000 篇文章
已處理 33000 篇文章
已處理 34000 篇文章
已處理 35000 篇文章
已處理 36000 篇文章
已處理 37000 篇文章
已處理 38000 篇文章
已處理 39000 篇文章
已處理 40000 篇文章
已處理 41000 篇文章
已處理 42000 篇文章
已處理 43000 篇文章
已處理 44000 篇文章
已處理 45000 篇文章
已處理 46000 篇文章
已處理 47000 篇文章
已處理 48000 篇文章
已處理 49000 篇文章
已處理 50000 篇文章
已處理 51000 篇文章
已處理 52000 篇文章
已處理 53000 篇文章
已處理 54000 篇文章
已處理 55000 篇文章
已處理 56000 篇文章
已處理 57000 篇文章
已處理 58000 篇文章
已處理 59000 篇文章
已處理 60000 篇文章
已處理 61000 篇文章
已處理 62000 篇文章
已處理 63000 篇文章
已處理 64000 篇文章
已處理 65000 篇文章
已處理 66000 篇文章
已處理 67000 篇文章
已處理 68000 篇文章
已處理 69000 篇文章
已處理 70000 篇文章
已處理 71000 篇文章
已處理 72000 篇文章
已

In [20]:
output = open('wiki_seg.txt', 'w', encoding='utf-8')
with open('wiki_zh_tw.txt', 'r', encoding='utf-8') as content :
    for texts_num, line in enumerate(content):
        line = line.strip('\n')
        words = jieba.cut(line, cut_all=False)
        for word in words:
#             if word not in stopword_set:
            output.write(word + ' ')
        output.write('\n')

        if (texts_num + 1) % 10000 == 0:
            print("已完成前 %d 行的斷詞" % (texts_num + 1))

已完成前 10000 行的斷詞
已完成前 20000 行的斷詞
已完成前 30000 行的斷詞
已完成前 40000 行的斷詞
已完成前 50000 行的斷詞
已完成前 60000 行的斷詞
已完成前 70000 行的斷詞
已完成前 80000 行的斷詞
已完成前 90000 行的斷詞
已完成前 100000 行的斷詞
已完成前 110000 行的斷詞
已完成前 120000 行的斷詞
已完成前 130000 行的斷詞
已完成前 140000 行的斷詞
已完成前 150000 行的斷詞
已完成前 160000 行的斷詞
已完成前 170000 行的斷詞
已完成前 180000 行的斷詞
已完成前 190000 行的斷詞
已完成前 200000 行的斷詞
已完成前 210000 行的斷詞
已完成前 220000 行的斷詞
已完成前 230000 行的斷詞
已完成前 240000 行的斷詞


In [58]:
MAX_SENTENCE_LENGTH = 50
WORD_VECTOR_DIM = 24

In [59]:
from gensim.test.utils import common_texts
from gensim.sklearn_api import W2VTransformer
from gensim import models
from gensim.models import word2vec
from gensim.models import Word2Vec
# Create a model to represent each word by a 10 dimensional vector.
sentences = models.word2vec.Text8Corpus(cut[0])

sentences = word2vec.LineSentence("wiki_seg.txt")
# wvmodel = word2vec.Word2Vec(sentences, size=250)
wvmodel = Word2Vec(sentences, size=WORD_VECTOR_DIM, window=5, min_count=0, workers=6)

In [287]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Activation, Flatten, Dropout, GRU, BatchNormalization, Bidirectional, Input
from keras.layers import Embedding, Concatenate, RepeatVector, Permute, Multiply, Lambda, Dot, Conv2D, MaxPooling2D, Reshape
from keras.models import Model
from keras import regularizers
import tensorflow as tf

In [61]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tokenizer = Tokenizer(nb_words=1000000000)
tokenizer.fit_on_texts(cut)
sequences = tokenizer.texts_to_sequences(cut)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

/home/daniel/.local/lib/python3.6/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 135685 unique tokens.


In [175]:
data = pad_sequences(sequences, maxlen=MAX_SENTENCE_LENGTH)

In [176]:
embeddings_index = {}
for word in wvmodel.wv.vocab:
    coefs = np.asarray(wvmodel.wv[word], dtype='float32')
    embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Found 141327 word vectors.


In [177]:
embedding_matrix = np.zeros((len(word_index) + 1, WORD_VECTOR_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [178]:
embedding_layer = Embedding(len(word_index) + 1,
                            WORD_VECTOR_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENTENCE_LENGTH,
                            trainable=False)

In [179]:
y = np.array(df['ratings'].astype(int))

In [180]:
for i in range(len(y)):
    y[i] = 1 if y[i] > 3 else 0

In [279]:
stratified_folder = StratifiedKFold(n_splits=3, random_state=0, shuffle=False) 
for train_index, test_index in stratified_folder.split(data, y):
    
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = y[train_index],y[test_index]
        
    input_layer = Input(shape=(X_train.shape[1],))
    
    embedded_layer = embedding_layer(input_layer)
    
    lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(32, return_sequences=True, return_state=True, activation='tanh'))(embedded_layer)
    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])
    
    attention = Dense(50, activation='softmax')(forward_h)
#     attention = Flatten()(attention)
#     attention = Activation('softmax')(attention)
#     attention = RepeatVector(128)(attention)
    lstm = Permute([2, 1])(lstm)

    sent_representation = Multiply()([lstm,attention])
#     sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(32,))(sent_representation)
    
    hidden_layer = Flatten()(sent_representation)

    hidden_layer = Dropout(0.5)(hidden_layer)

    output_layer = Dense(1,activation='sigmoid')(hidden_layer)

    ml_model = Model(inputs=input_layer, outputs=[output_layer])
    ml_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    ml_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, verbose=1, batch_size=1024)
    
    models.append(ml_model)
    pred = ml_model.predict(X_test)>0.5
    
    print(f1_score(pred, y_test, average='macro'))
    print(confusion_matrix(pred, y_test))
    
    break

Train on 162186 samples, validate on 81094 samples
Epoch 1/15
162186/162186 [==============================] - 27s 169us/step - loss: 0.4465 - acc: 0.7955 - val_loss: 0.3235 - val_acc: 0.8582
Epoch 2/15
162186/162186 [==============================] - 22s 133us/step - loss: 0.3199 - acc: 0.8592 - val_loss: 0.3049 - val_acc: 0.8666
Epoch 3/15
162186/162186 [==============================] - 20s 124us/step - loss: 0.3078 - acc: 0.8642 - val_loss: 0.2982 - val_acc: 0.8698
Epoch 4/15
162186/162186 [==============================] - 21s 129us/step - loss: 0.3004 - acc: 0.8686 - val_loss: 0.2955 - val_acc: 0.8717
Epoch 5/15
162186/162186 [==============================] - 21s 127us/step - loss: 0.2956 - acc: 0.8713 - val_loss: 0.2909 - val_acc: 0.8738
Epoch 6/15
162186/162186 [==============================] - 21s 132us/step - loss: 0.2913 - acc: 0.8728 - val_loss: 0.2872 - val_acc: 0.8749
Epoch 7/15
 82944/162186 [==============>...............] - ETA: 9s - loss: 0.2870 - acc: 0.8750

KeyboardInterrupt: 

In [321]:
num_filters=64
filter_sizes = [3,3,3]
stratified_folder = StratifiedKFold(n_splits=3, random_state=0, shuffle=False) 
for train_index, test_index in stratified_folder.split(data, y):
    
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = y[train_index],y[test_index]
     
    input_layer = Input(shape=(X_train.shape[1],))
    
    embedded_layer = embedding_layer(input_layer)
    
    reshape = Reshape((MAX_SENTENCE_LENGTH,WORD_VECTOR_DIM, 1))(embedded_layer)

    conv_0 = Conv2D(num_filters, (filter_sizes[0], WORD_VECTOR_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
    conv_1 = Conv2D(num_filters, (filter_sizes[1], WORD_VECTOR_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
    conv_2 = Conv2D(num_filters, (filter_sizes[2], WORD_VECTOR_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

    maxpool_0 = MaxPooling2D((12, 1), strides=(1,1))(conv_0)
    maxpool_1 = MaxPooling2D((12, 1), strides=(1,1))(conv_1)
    maxpool_2 = MaxPooling2D((12, 1), strides=(1,1))(conv_2)

#     conv_0 = Conv2D(num_filters, (filter_sizes[0], WORD_VECTOR_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(maxpool_0)
#     conv_1 = Conv2D(num_filters, (filter_sizes[1], WORD_VECTOR_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(maxpool_1)
#     conv_2 = Conv2D(num_filters, (filter_sizes[2], WORD_VECTOR_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(maxpool_2)
    
#     maxpool_0 = MaxPooling2D((2, 1), strides=(1,1))(conv_0)
#     maxpool_1 = MaxPooling2D((2, 1), strides=(1,1))(conv_1)
#     maxpool_2 = MaxPooling2D((2, 1), strides=(1,1))(conv_2)
    
    merged_tensor = Concatenate()([maxpool_0, maxpool_1, maxpool_2])
    flatten = Flatten()(merged_tensor)
#     reshape = Reshape((3*num_filters,))(flatten)
    dropout = Dropout(0.5)(flatten) 
    dropout = Dense(1024)(flatten)
    output = Dense(units=1, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01))(dropout)
    
    ml_model = Model(inputs=input_layer, outputs=output)
    ml_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    ml_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, verbose=1, batch_size=1024)

Train on 162186 samples, validate on 81094 samples
Epoch 1/15
162186/162186 [==============================] - 47s 292us/step - loss: 0.6224 - acc: 0.8197 - val_loss: 0.3566 - val_acc: 0.8564
Epoch 2/15
162186/162186 [==============================] - 42s 256us/step - loss: 0.3480 - acc: 0.8572 - val_loss: 0.3413 - val_acc: 0.8565
Epoch 3/15
162186/162186 [==============================] - 40s 249us/step - loss: 0.3308 - acc: 0.8621 - val_loss: 0.3252 - val_acc: 0.8627
Epoch 4/15
162186/162186 [==============================] - 42s 256us/step - loss: 0.3243 - acc: 0.8636 - val_loss: 0.3217 - val_acc: 0.8646
Epoch 5/15
162186/162186 [==============================] - 44s 268us/step - loss: 0.3189 - acc: 0.8652 - val_loss: 0.3161 - val_acc: 0.8664
Epoch 6/15
162186/162186 [==============================] - 44s 270us/step - loss: 0.3173 - acc: 0.8671 - val_loss: 0.3201 - val_acc: 0.8665
Epoch 7/15
162186/162186 [==============================] - 41s 253us/step - loss: 0.3149 - acc: 0.8680

KeyboardInterrupt: 

In [255]:
pred = ml_model.predict(X_test)>0.5

print(f1_score(pred, y_test, average='macro'))
print(confusion_matrix(pred, y_test))

0.8470822365189703
[[17146  4432]
 [ 5390 54126]]


In [218]:
import pickle
ml_model.save("api_test.h5")
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [168]:
test = ['可以爛到這樣也是很猛']*10000

In [169]:
test = list(map(lambda x : " ".join(jieba.cut(x.strip() + " $ENDING$")), test))

In [170]:
test_sequences = tokenizer.texts_to_sequences(test)

In [171]:
data = pad_sequences(test_sequences, maxlen=MAX_SENTENCE_LENGTH)

In [172]:
ml_model.predict_proba(data)

array([[0.37695],
       [0.37695],
       [0.37695],
       ...,
       [0.37695],
       [0.37695],
       [0.37695]], dtype=float32)

In [232]:
import requests

<Response [201]>

In [ ]:
ml_model.predict_proba(X_test)

In [82]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.naive_bayes import MultinomialNB, ComplementNB
import lightgbm as lgbm

vectorizer = CountVectorizer(max_df=0.3, min_df=0).fit(cut)
counts_vector = vectorizer.transform(cut)

tfidfVectorizer = TfidfTransformer(smooth_idf=True).fit(counts_vector)
tfidf_vector = tfidfVectorizer.transform(counts_vector)

In [83]:
stratified_folder = StratifiedKFold(n_splits=4, random_state=0, shuffle=False) 
for train_index, test_index in stratified_folder.split(tfidf_vector, y): 
    X_train, X_test = tfidf_vector[train_index], tfidf_vector[test_index]
    y_train, y_test = y[train_index], y[test_index] 
    
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'binary_logloss',
        'num_classes':1,
        'subsample': .9,
        'colsample_bytree': .5,
        'reg_alpha': .1,
        'reg_lambda': .1,
        'min_split_gain': 0.001,
        'min_child_weight': 5,
        'silent':True,
        'verbosity':-1,
        'n_jobs':6
    }

    clf = lgbm.LGBMClassifier(**lgb_params,learning_rate=0.05,
                                    n_estimators=10000,max_depth=4)
    clf.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train),(X_test, y_test)],
        verbose=50,
        early_stopping_rounds=50
    )
    pred = clf.predict(X_test, num_iteration=clf.best_iteration_)
    print(f1_score(pred, y_test, average='macro'))
    print(confusion_matrix(pred, y_test))

Training until validation scores don't improve for 50 rounds.
[50]	training's binary_logloss: 0.480827	valid_1's binary_logloss: 0.484316
[100]	training's binary_logloss: 0.447251	valid_1's binary_logloss: 0.451732
[150]	training's binary_logloss: 0.42897	valid_1's binary_logloss: 0.434317
[200]	training's binary_logloss: 0.416446	valid_1's binary_logloss: 0.422571
[250]	training's binary_logloss: 0.406927	valid_1's binary_logloss: 0.413702
[300]	training's binary_logloss: 0.399239	valid_1's binary_logloss: 0.406634
[350]	training's binary_logloss: 0.392771	valid_1's binary_logloss: 0.400777
[400]	training's binary_logloss: 0.387237	valid_1's binary_logloss: 0.395756
[450]	training's binary_logloss: 0.382374	valid_1's binary_logloss: 0.391295
[500]	training's binary_logloss: 0.378057	valid_1's binary_logloss: 0.387533
[550]	training's binary_logloss: 0.374192	valid_1's binary_logloss: 0.384128
[600]	training's binary_logloss: 0.370619	valid_1's binary_logloss: 0.381029
[650]	training's

[5300]	training's binary_logloss: 0.290589	valid_1's binary_logloss: 0.323912
[5350]	training's binary_logloss: 0.290286	valid_1's binary_logloss: 0.323771
[5400]	training's binary_logloss: 0.289982	valid_1's binary_logloss: 0.323643
[5450]	training's binary_logloss: 0.289677	valid_1's binary_logloss: 0.323542
[5500]	training's binary_logloss: 0.289377	valid_1's binary_logloss: 0.323424
[5550]	training's binary_logloss: 0.289097	valid_1's binary_logloss: 0.323329
[5600]	training's binary_logloss: 0.288811	valid_1's binary_logloss: 0.323181
[5650]	training's binary_logloss: 0.288537	valid_1's binary_logloss: 0.323048


KeyboardInterrupt: 

In [ ]:
clf.predict_proba(X_test)

In [ ]:
test = '爛'
test = " ".join(jieba.cut(test))
test = tfidfVectorizer.transform(vectorizer.transform([test]))
model.predict_proba(test)